# データ分析とモデル作成sample

## 内容

- タイタニックのデータを使用して，データ分析とモデル作成を行う

### 各カラムの意味

1. **survived**: 乗客が生存したかどうか（0 = 死亡, 1 = 生存）。
2. **pclass**: 乗客のチケットクラス（1 = 1等, 2 = 2等, 3 = 3等）。
3. **sex**: 乗客の性別（male = 男性, female = 女性）。
4. **age**: 乗客の年齢。NaNの値も含まれ、年齢が不明な場合があります。
5. **sibsp**: 兄弟姉妹または配偶者の数。乗船した兄弟姉妹または配偶者の数を示します。
6. **parch**: 両親または子供の数。乗船した親または子供の数を示します。
7. **fare**: 乗船料金。乗客が支払った運賃を表します。
8. **embarked**: 乗船港（C = Cherbourg, Q = Queenstown, S = Southampton）。
9. **class**: チケットのクラスを文字列で示したもの（'First', 'Second', 'Third'）。
10. **who**: 乗客のカテゴリー（'man', 'woman', 'child'）。
11. **adult_male**: 乗客が成人男性かどうか（True = 成人男性, False = それ以外）。
12. **deck**: 乗客が乗っていたデッキ（甲板）のレベル。NaNの値も多く含まれます。
13. **embark_town**: 乗船した港の町（'Cherbourg', 'Queenstown', 'Southampton'）。
14. **alive**: 生存か死亡かを文字列で示したもの（'yes' = 生存, 'no' = 死亡）。
15. **alone**: 乗客が単独で乗船したかどうか（True = 単独, False = 家族や他の人と一緒）。

# Load modules
- moduleのloadを行う.

In [ ]:
!pip install japanize_matplotlib
!pip install optuna optuna_integration
!pip install shap

In [ ]:
# ライブラリ読み込み
import sys, os
import time
import gc
from datetime import datetime as dt
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import japanize_matplotlib

import joblib
import re # 正規表現

import seaborn as sns

from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV

# 評価関数
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [ ]:
# LightGBM
import lightgbm as lgb

# optunaによるLightGBM
import optuna.integration.lightgbm as optuna_lgb

# SHAP
import shap

In [ ]:
gc.collect()

# Configure

## Google Colab用設定

In [ ]:
# 各自書き換えてください
current_project_dpath = '/content/drive/MyDrive/work/matsue_ct/20240831_データ分析研修'
print(current_project_dpath)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
os.chdir(current_project_dpath)

# 現在のディレクトリを確認
print(os.getcwd())

## pandas.DataFrameの表示行数・列数を変更

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## warningの表示を削除

In [ ]:
# warningの削除
import warnings
warnings.filterwarnings('ignore')

## random seed
- random系moduleのseed値を設定する.

In [ ]:
# random系moduleのseed値を設定
random.seed(57)
np.random.seed(57)

# Constants

## date

In [ ]:
today_dt = dt.today()
today_str = dt.strftime(today_dt, '%Y%m%d')
today_str

## paths

In [ ]:
input_dpath = './input/'
output_dpath = './output/'

# Functions

In [ ]:
def logistic(x):
    """
    logistic関数を計算します。

    logistic関数は、ロジスティック回帰などで使用されるシグモイド関数であり、
    入力の実数値を0から1の範囲の値にマッピングします。

    パラメータ
    ----------
    x : float または np.ndarray
        入力値。実数または実数の配列。

    戻り値
    -------
    float または np.ndarray
        入力xに対するlogistic関数の値。
    """
    return 1 / (1 + np.exp(-x))

In [ ]:
def logit(p):
    """
    logit関数を計算します。

    logit関数は、ロジスティック関数（シグモイド関数）の逆関数であり、確率をオッズ比に変換します。
    確率pは0から1の範囲内である必要があります。

    パラメータ
    ----------
    p : float または np.ndarray
        確率値（0から1の範囲内）。

    戻り値
    -------
    float または np.ndarray
        入力pに対するlogit値。
    """
    # 安全性チェック：0と1の値が入力されると、logが無限大になるため小さな値を追加
    p = np.clip(p, 1e-15, 1 - 1e-15)

    return np.log(p / (1 - p))


# Load data

## 前処理済み中間データ

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load(f'{input_dpath}pp_titanic_data_dict.pkl3')

In [ ]:
type(pp_data_dict)

### dictからデータを読み取る

In [ ]:
train_x = pp_data_dict['train_x']
train_y = pp_data_dict['train_y']
test_x = pp_data_dict['test_x']
test_y = pp_data_dict['test_y']

In [ ]:
# shapeの確認
train_x.shape, train_y.shape, test_x.shape, test_x.shape

# LightGBM

## set vars

In [ ]:
# 説明変数
features = train_x.columns.tolist()

# 目的変数
target = 'survived'

In [ ]:
len(features)

In [ ]:
features[:5]

In [ ]:
features

## 一部前処理

### ハイパーパラメータ用にデータを分ける

In [ ]:
# 7:3の割合でホールドアウト法を行う.
train_train_x, train_valid_x, train_train_y, train_valid_y = train_test_split(
    train_x, train_y,
    test_size=0.3,
    random_state=57,
    shuffle=True
)

In [ ]:
train_train_x.shape, train_train_y.shape, train_valid_x.shape, train_valid_y.shape

### LightGBM向けにデータセットを作成する

In [ ]:
# LightGBM用データセットを生成する
lgb_train_train_dataset = lgb.Dataset(
    train_train_x,
    train_train_y
)
lgb_train_valid_dataset = lgb.Dataset(
    train_valid_x,
    train_valid_y,
    reference=lgb_train_train_dataset
)

In [ ]:
# LightGBM用データセットを生成する
lgb_train_dataset = lgb.Dataset(
    train_x,
    train_y
)

## Optunaによるハイパーパラメータチューニング

In [ ]:
# ハイパーパラメータ
lgbc_params = {
    # 問題設定: 2値分類
    'objective': 'binary',

    # 評価関数: AUC
    'metric': 'auc',
    'verbosity': -1,
}

In [ ]:
# モデルの学習を行う.
optuna_lgbc = optuna_lgb.train(
    lgbc_params, # ハイパーパラメータ
    train_set=lgb_train_train_dataset, # 学習データ
    num_boost_round=100, # boostingを行う回数
    valid_sets=lgb_train_valid_dataset, # 検証データ
    verbosity=20 # boosting20回に1回結果出力
)

In [ ]:
# 最適なパラメータの確認
best_lgbc_params = optuna_lgbc.params
best_lgbc_params

## create model

In [ ]:
# モデルの学習を行う.
lgbc = lgb.train(
    best_lgbc_params, # 最適なハイパーパラメータ
    train_set=lgb_train_dataset, # 学習データ
    num_boost_round=100 # boostingを行う回数
)

## 精度評価

### train

In [ ]:
# train予測
# lgbm本家ではpredictが確率になっている
lgbc_prob_train_y = lgbc.predict(train_x)

# 確率が0.5以上の時1と判定する.
lgbc_pred_train_y = np.where(
    lgbc_prob_train_y >= 0.5,
    1,
    0
)

In [ ]:
lgbc_train_valid_df = pd.DataFrame(
    {
        'prob_y': lgbc_prob_train_y,
        'pred_y': lgbc_pred_train_y,
        target: train_y
    }
)
lgbc_train_valid_df.head()

In [ ]:
# accuracy
lgbc_train_accuracy_val = accuracy_score(
    lgbc_train_valid_df[target],
    lgbc_train_valid_df['pred_y']
)

# auc
lgbc_train_auc_val = roc_auc_score(
    lgbc_train_valid_df[target],
    lgbc_train_valid_df['prob_y']
)

print('accuracy:', lgbc_train_accuracy_val)
print('auc:', lgbc_train_auc_val)

### test

In [ ]:
# test予測
# lgbm本家ではpredictが確率になっている
lgbc_prob_test_y = lgbc.predict(test_x)

# 確率が0.5以上の時1と判定する.
lgbc_pred_test_y = np.where(
    lgbc_prob_test_y >= 0.5,
    1,
    0
)

In [ ]:
lgbc_test_valid_df = pd.DataFrame(
    {
        'prob_y': lgbc_prob_test_y,
        'pred_y': lgbc_pred_test_y,
        target: test_y
    }
)
lgbc_test_valid_df.head()

In [ ]:
# accuracy
lgbc_test_accuracy_val = accuracy_score(
    lgbc_test_valid_df[target],
    lgbc_test_valid_df['pred_y']
)

# auc
lgbc_test_auc_val = roc_auc_score(
    lgbc_test_valid_df[target],
    lgbc_test_valid_df['prob_y']
)

print('accuracy:', lgbc_test_accuracy_val)
print('auc:', lgbc_test_auc_val)

### 決定木・ランダムフォレストとの比較

- 基本的にLightGBMが最も精度が良い

In [ ]:
print('lgbc train accuracy:', lgbc_train_accuracy_val)
print('lgbc test accuracy:', lgbc_test_accuracy_val)
print('=' * 60)
print('lgbc train auc:', lgbc_train_auc_val)
print('lgbc test auc:', lgbc_test_auc_val)

# SHAP

In [ ]:
# init javascript
shap.initjs()

## explainer作成

In [ ]:
# explainer作成
explainer = shap.TreeExplainer(
    model=lgbc,
    # model_output='margin'
)

## shap_values計算

In [ ]:
shap_values = explainer.shap_values(
    train_x,
    check_additivity=False
)

## summary_plot

In [ ]:
# summary plot(bar)
shap.summary_plot(
    shap_values,
    train_x,
    plot_type='bar'
)

In [ ]:
# summary plot
shap.summary_plot(
    shap_values,
    train_x
)

## dependence_plot

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(ls='--')

# dependence
shap.dependence_plot(
    ind='fare',
    shap_values=shap_values,
    features=train_x,
    feature_names=features,
    ax=ax
)

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(ls='--')

# dependence
shap.dependence_plot(
    ind='age',
    shap_values=shap_values,
    features=train_x,
    feature_names=features,
    ax=ax
)

## force_plot

### idx:1

In [ ]:
target_row_index = 1
display(pd.DataFrame(train_x.iloc[target_row_index]).T)
print('survived:', train_y.iloc[target_row_index])

In [ ]:
# force plot
shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values[target_row_index,:],
    features=train_x.iloc[target_row_index,:],
    feature_names=features
)

## waterfall

In [ ]:
# waterfall用のexplainer
explainer_waterfall = shap.TreeExplainer(model=lgbc)

# .shap_values不使用
shap_values_waterfall = explainer_waterfall(
    train_x,
    check_additivity=False
)

In [ ]:
pd.concat([train_y, train_x], axis=1).reset_index(drop=True).head(10)

In [ ]:
pd.concat([train_y, train_x], axis=1).reset_index(drop=True).tail(10)

### idx:1

In [ ]:
target_row_index = 1
display(pd.DataFrame(train_x.iloc[target_row_index]).T)
print('survived:', train_y.iloc[target_row_index])

In [ ]:
# 変数表示数
max_display_num = 15

# waterfall
shap.waterfall_plot(
    shap_values=shap_values_waterfall[target_row_index,:],
    max_display=max_display_num,
    show=True
)

In [ ]:
temp_shap_values = shap_values_waterfall[target_row_index,:]
temp_shap_values

In [ ]:
temp_shap_values.values.sum() + temp_shap_values.base_values

### idx:495

In [ ]:
target_row_index = 495
display(pd.DataFrame(train_x.iloc[target_row_index]).T)
print('survived:', train_y.iloc[target_row_index])

In [ ]:
# 変数表示数
max_display_num = 15

# waterfall
shap.waterfall_plot(
    shap_values=shap_values_waterfall[target_row_index,:],
    max_display=max_display_num,
    show=True
)

In [ ]:
temp_shap_values = shap_values_waterfall[target_row_index,:]
temp_shap_values

In [ ]:
temp_shap_values.values.sum() + temp_shap_values.base_values

In [ ]:
train_y.mean(), logit(train_y.mean())

In [ ]:
lgbc_prob_train_y.mean(), logit(lgbc_prob_train_y.mean())

# notebookをhtml化

In [ ]:
!jupyter nbconvert --to html ./src/20240831_LightGBMとSHAPの使用_colab.ipynb